In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, WebDriverException
import os

# Chrome 옵션 설정 (headless 모드, user-agent 설정 등 필요에 따라 추가)
options = Options()
# options.add_argument('--headless')  # headless 모드로 실행 (GUI 없이 실행)
# options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

# Service 객체 생성 - executable_path 인자 없이 생성 (PATH 환경 변수 활용)
service = Service() # executable_path 생략
driver = webdriver.Chrome(service=service, options=options)

url = 'https://www.card-gorilla.com/chart/top100?term=weekly'

try:
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    cards_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//ul[@class='rk_lst']/li")))

    # 데이터를 저장할 리스트 초기화
    rankings = []
    card_names = []
    corp_names = []
    benefits = []
    detail_urls = []
    image_urls = []


    for card in cards_elements:
        if "ad" not in card.get_attribute("class"):
            try: # 각 카드 정보 추출 시 오류 처리
                rank_element = card.find_element(By.XPATH, ".//div[@class='winner']") if "winner" in card.get_attribute("class") else card.find_element(By.XPATH, ".//div[@class='num']")

                if "winner" in card.get_attribute("class"):  # 1위 카드 처리
                    card_name = card.find_element(By.XPATH, ".//span[@class='card_name']").text.split('\n')[0]
                    corp_name = card.find_element(By.XPATH, ".//p[@class='corp_name']").text
                else:
                    card_name_element = card.find_element(By.XPATH, ".//p[@class='card_name']")
                    card_name = card_name_element.text.split('\n')[0]
                    corp_name = ' '.join(card.find_element(By.XPATH, ".//p[@class='corp_name']").text.split())

                detail_url = card.find_element(By.XPATH, ".//a").get_attribute('href')
                img_url = card.find_element(By.XPATH, ".//div[@class='card_img']//img").get_attribute('src')

                # 수집한 데이터 저장
                rankings.append(rank_element.text)
                card_names.append(card_name)
                corp_names.append(corp_name)
                detail_urls.append(detail_url)
                image_urls.append(img_url)

                benefits.append([])

            except Exception as card_error: # 카드 정보 추출 중 오류 발생 시 로그 출력 및 건너뛰기
                print(f"카드 정보 추출 중 오류 발생: {card_error}")
                print(f"오류 발생 카드 Element (HTML): {card.get_attribute('outerHTML')}") # 문제 발생 카드 HTML 출력
                continue # 오류 발생 카드 건너뛰고 다음 카드 처리


    # 상세 페이지로의 접근 및 혜택 정보 수집
    for idx, url in enumerate(detail_urls):
        if not isinstance(url, str):  # URL이 문자열이 아니면 건너뛰기
            print(f"유효하지 않은 URL: {url}")
            continue

        try:
            driver.get(url)
            benefit_area = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='lst bene_area']")))
            benefit_texts_detail_page = []
            p_elements = benefit_area.find_elements(By.XPATH, ".//p")
            i_elements = benefit_area.find_elements(By.XPATH, ".//i")

            for p_element in p_elements:
                benefit_texts_detail_page.append(p_element.text)
            for i_element in i_elements:
                benefit_texts_detail_page.append(i_element.text)


            benefits[idx] = benefit_texts_detail_page
        except TimeoutException:
            print(f"혜택 정보를 얻지 못한 URL (Timeout): {url}")
        except Exception as detail_error: # 상세 페이지 오류 처리
            print(f"상세 페이지 오류 발생: {detail_error}")
            print(f"오류 발생 URL: {url}")


    # 혜택 정보 정리
    benefits_cleaned = []
    for benefit_list in benefits:
        cleaned_list = []
        for benefit in benefit_list:
            cleaned_benefit = ' '.join(benefit.split())
            cleaned_list.append(cleaned_benefit)
        benefits_cleaned.append(cleaned_list)

    # 수집한 데이터를 pandas DataFrame으로 변환
    df = pd.DataFrame({
        'Ranking': rankings,
        'Card Name': card_names,
        'Corporate Name': corp_names,
        'Benefits': benefits_cleaned,
        'Image URLs': image_urls
    })

    # DataFrame을 Excel 파일로 저장
    print(df)
    df.to_excel(r"C:\Users\사용자이름\Downloads\credit_card_gorilla_data.xlsx", index=False)
    print("Excel 파일 저장 완료!")


except TimeoutException:
    print("웹 페이지 로딩 시간 초과")
except Exception as e: # 전체 코드 실행 중 예외 처리
    print(f"예상치 못한 오류 발생: {e}")

finally: # finally 블록에서 driver.quit() 보장
    if 'driver' in locals(): # driver 변수가 정의되었는지 확인 후 종료
        driver.quit()

유효하지 않은 URL: None
   Ranking                        Card Name Corporate Name  \
0                                                            
1        2                     신한카드 Mr.Life           신한카드   
2        3                   LOCA LIKIT 1.2           롯데카드   
3        4  삼성카드 & MILEAGE PLATINUM (스카이패스)           삼성카드   
4        5                           현대카드 M           현대카드   
..     ...                              ...            ...   
95      96                티머니 Pay & GO 신한카드           신한카드   
96      97                      신한카드 구독 좋아요           신한카드   
97      98                       대한항공카드 120           현대카드   
98      99                   토스뱅크 하나카드 Wide           토스뱅크   
99     100             메리어트 본보이™ 더 베스트 신한카드           신한카드   

                                             Benefits  \
0   [선택형, 대중교통, 통신, 영화, 해외, 유의사항, 라이프스타일 패키지 선택, 대...   
1   [공과금, 편의점, 병원/약국, 생활, 온라인쇼핑, 택시, 푸드, 대형마트, 주유소...   
2   [모든가맹점, 온라인쇼핑, 유의사항, 어디서나 1.2% 결제일 할인 국내 모든 가맹...   
3   [대한항공

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, WebDriverException
import os

# Chrome 옵션 설정 (headless 모드, user-agent 설정 등 필요에 따라 추가)
options = Options()
# options.add_argument('--headless')  # headless 모드로 실행 (GUI 없이 실행)
# options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

# Service 객체 생성 - executable_path 인자 없이 생성 (PATH 환경 변수 활용)
service = Service() # executable_path 생략
driver = webdriver.Chrome(service=service, options=options)

url = 'https://www.card-gorilla.com/chart/check100'

try:
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    cards_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//ul[@class='rk_lst']/li")))

    # 데이터를 저장할 리스트 초기화
    rankings = []
    card_names = []
    corp_names = []
    benefits = []
    detail_urls = []
    image_urls = []


    for card in cards_elements:
        if "ad" not in card.get_attribute("class"):
            try: # 각 카드 정보 추출 시 오류 처리
                rank_element = card.find_element(By.XPATH, ".//div[@class='winner']") if "winner" in card.get_attribute("class") else card.find_element(By.XPATH, ".//div[@class='num']")

                if "winner" in card.get_attribute("class"):  # 1위 카드 처리
                    card_name = card.find_element(By.XPATH, ".//span[@class='card_name']").text.split('\n')[0]
                    corp_name = card.find_element(By.XPATH, ".//p[@class='corp_name']").text
                else:
                    card_name_element = card.find_element(By.XPATH, ".//p[@class='card_name']")
                    card_name = card_name_element.text.split('\n')[0]
                    corp_name = ' '.join(card.find_element(By.XPATH, ".//p[@class='corp_name']").text.split())

                detail_url = card.find_element(By.XPATH, ".//a").get_attribute('href')
                img_url = card.find_element(By.XPATH, ".//div[@class='card_img']//img").get_attribute('src')

                # 수집한 데이터 저장
                rankings.append(rank_element.text)
                card_names.append(card_name)
                corp_names.append(corp_name)
                detail_urls.append(detail_url)
                image_urls.append(img_url)

                benefits.append([])

            except Exception as card_error: # 카드 정보 추출 중 오류 발생 시 로그 출력 및 건너뛰기
                print(f"카드 정보 추출 중 오류 발생: {card_error}")
                print(f"오류 발생 카드 Element (HTML): {card.get_attribute('outerHTML')}") # 문제 발생 카드 HTML 출력
                continue # 오류 발생 카드 건너뛰고 다음 카드 처리


    # 상세 페이지로의 접근 및 혜택 정보 수집
    for idx, url in enumerate(detail_urls):
        if not isinstance(url, str):  # URL이 문자열이 아니면 건너뛰기
            print(f"유효하지 않은 URL: {url}")
            continue

        try:
            driver.get(url)
            benefit_area = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='lst bene_area']")))
            benefit_texts_detail_page = []
            p_elements = benefit_area.find_elements(By.XPATH, ".//p")
            i_elements = benefit_area.find_elements(By.XPATH, ".//i")

            for p_element in p_elements:
                benefit_texts_detail_page.append(p_element.text)
            for i_element in i_elements:
                benefit_texts_detail_page.append(i_element.text)


            benefits[idx] = benefit_texts_detail_page
        except TimeoutException:
            print(f"혜택 정보를 얻지 못한 URL (Timeout): {url}")
        except Exception as detail_error: # 상세 페이지 오류 처리
            print(f"상세 페이지 오류 발생: {detail_error}")
            print(f"오류 발생 URL: {url}")


    # 혜택 정보 정리
    benefits_cleaned = []
    for benefit_list in benefits:
        cleaned_list = []
        for benefit in benefit_list:
            cleaned_benefit = ' '.join(benefit.split())
            cleaned_list.append(cleaned_benefit)
        benefits_cleaned.append(cleaned_list)

    # 수집한 데이터를 pandas DataFrame으로 변환
    df = pd.DataFrame({
        'Ranking': rankings,
        'Card Name': card_names,
        'Corporate Name': corp_names,
        'Benefits': benefits_cleaned,
        'Image URLs': image_urls
    })

    # DataFrame을 Excel 파일로 저장
    print(df)
    df.to_excel(r"C:\Users\사용자이름\Downloads\check_card_gorilla_data.xlsx", index=False)
    print("Excel 파일 저장 완료!")


except TimeoutException:
    print("웹 페이지 로딩 시간 초과")
except Exception as e: # 전체 코드 실행 중 예외 처리
    print(f"예상치 못한 오류 발생: {e}")

finally: # finally 블록에서 driver.quit() 보장
    if 'driver' in locals(): # driver 변수가 정의되었는지 확인 후 종료
        driver.quit()

유효하지 않은 URL: None
   Ranking                 Card Name Corporate Name  \
0                                                     
1        2          노리2 체크카드(KB Pay)         KB국민카드   
2        3  신한카드 Deep Dream 체크(미니언즈)           신한카드   
3        4             카카오페이 신한 체크카드           신한카드   
4        5          네이버페이 머니 하나 체크카드           하나카드   
..     ...                       ...            ...   
95      96     케이뱅크 X KT멤버십더블혜택 체크카드           케이뱅크   
96      97              카카오뱅크 mini카드          카카오뱅크   
97      98              국민행복 체크카드 S2           우리카드   
98      99       SC제일은행-현대카드 M CHECK           현대카드   
99     100                  내일배움체크카드         NH농협카드   

                                             Benefits  \
0   [대중교통, 대중교통, 캐시백, 캐시백, 캐시백, 기타, 유의사항, [K-패스] 매...   
1   [카페, 게임, 공연/전시, 드럭스토어, 편의점, 디지털구독, 배달앱, 통신, 영화...   
2   [적립, 생활, 생활, 주유소, 택시, 적립, 유의사항, [모두드림] 국내/외 일시...   
3   [선택형, 카카오페이, 대중교통, 통신, 영화, 해외, 프리미엄 서비스, 유의사항,...   
4   [적립, 선택형, 유의사항, 네이버페이 포인트 적립 서비스